## Бонусное задание: word2vec на PyTorch

Как вы уже могли заметить, идея, лежащая в основе [word2vec](https://arxiv.org/pdf/1310.4546), достаточно общая. В данном задании вы реализуете его самостоятельно.

Дисклеймер: не стоит удивляться тому, что реализация от `gensim` (или аналоги) обучается быстрее и работает точнее. Она использует множество доработок и ускорений, а также достаточно эффективный код. Ваша задача добиться промежуточных результатов за разумное время.

P.s. Как ни странно, GPU в этом задании нам не потребуется.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk bokeh umap-learn```


In [ ]:
pip install --upgrade nltk bokeh umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.4.3
    Uninstalling bokeh-3.4.3:
      Successfully uninstalled bokeh-3.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 1.4.5 requires bokeh<3.5.0,>=3.4.0, but you have bokeh 3.6.0 which is incompatible.


In [ ]:
pip install umap

  Preparing metadata (setup.py) ... done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3542 sha256=1f7c2e770cc04fa2ab760fc246eab9bc080a3e27eef6e5348cf1846eaa32c5b8
  Stored in directory: /root/.cache/pip/wheels/15/f1/28/53dcf7a309118ed35d810a5f9cb995217800f3f269ab5771cb
Successfully built umap


In [ ]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2024-10-30 13:02:35--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-10-30 13:02:35--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./quora.txt’

./quora.txt             [  <=>               ] 164.92K   399KB/s    in 0.4s    

2024-10-30 13:02:36 (399 KB/s) - ‘./quora.txt’ saved [168881]



In [ ]:
data = list(open('/content/quora (1).txt', encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

Токенизация – первый шаг.
Тексты, с которыми мы работаем, включают в себя пунктуацию, смайлики и прочие нестандартные токены, так что простой `str.split` не подойдет.

Обратимся к `nltk` - библиотеку, нашла широкое применеие в области NLP.

In [ ]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


Несколько проверок:

In [ ]:
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
]
data_tok = [x for x in data_tok if len(x) >= 3]

In [ ]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

Ниже заданы константы ширины окна контекста и проведена предобработка для построения skip-gram модели.

In [ ]:
min_count = 5
window_radius = 5

In [ ]:
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))

In [ ]:

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if counter >= min_count:
        word_count_dict[word] = counter

vocabulary = set(word_count_dict.keys())
del vocabulary_with_counter

In [ ]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {index: word for word, index in word_to_index.items()}

In [ ]:
len(index_to_word)

28546

Пары `(слово, контекст)` на основе доступного датасета сгенерированы ниже.

In [ ]:
context_pairs = []

for text in data_tok:
    for i, central_word in enumerate(text):
        context_indices = range(
            max(0, i - window_radius), min(i + window_radius, len(text))
        )
        for j in context_indices:
            if j == i:
                continue
            context_word = text[j]
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append(
                    (word_to_index[central_word], word_to_index[context_word])
                )

print(f"Generated {len(context_pairs)} pairs of target and context words.")

Generated 40220313 pairs of target and context words.


#### Подзадача №1: subsampling
Для того, чтобы сгладить разницу в частоте встречаемсости слов, необходимо реализовать механизм subsampling'а.
Для этого вам необходимо реализовать функцию ниже.

Вероятность **исключить** слово из обучения (на фиксированном шаге) вычисляется как
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
где $f(w_i)$ – нормированная частота встречаемости слова, а $t$ – заданный порог (threshold).

In [ ]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """

    ### YOUR CODE HERE
    total_word_count = sum(word_count_dict.values())
    keep_prob_dict = {
      word: (threshold / (freq / total_word_count)) ** 0.5
      for word, freq in word_count_dict.items()}

    return keep_prob_dict

In [ ]:
word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
subsample_frequent_words(word_counts)
    # {'the': 0.028, 'is': 0.223, 'apple': 1.0}

{'the': 0.0034785054261852176,
 'is': 0.007778174593052023,
 'apple': 0.034785054261852175}

#### Подзадача №2: negative sampling
Для более эффективного обучения необходимо не только предсказывать высокие вероятности для слов из контекста, но и предсказывать низкие для слов, не встреченных в контексте. Для этого вам необходимо вычислить вероятност использовать слово в качестве negative sample, реализовав функцию ниже.

В оригинальной статье предлагается оценивать вероятность слов выступать в качестве negative sample согласно распределению $P_n(w)$
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

где $U(w)$ распределение слов по частоте (или, как его еще называют, по униграммам), а $Z$ – нормировочная константа, чтобы общая мера была равна $1$.

In [ ]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """

    ### YOUR CODE HERE
    adj_pr = { word: count**0.75 for word, count in word_count_dict.items()}

    total_fr = sum(adj_pr.values())



    negative_sampling_prob_dict = {
        word: freq / total_fr for word, freq in adj_pr.items()}   # THIS IS A PLACEHOLDER!
    return negative_sampling_prob_dict

In [ ]:
word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
get_negative_sampling_prob(word_counts)
    # {'the': 0.298, 'is': 0.160, 'apple': 0.042}

{'the': 0.751488398196177,
 'is': 0.2247474520689081,
 'apple': 0.023764149734914898}

Для удобства, преобразуем полученные словари в массивы (т.к. все слова все равно уже пронумерованы).

In [ ]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

In [ ]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

In [ ]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

Если все прошло успешно, функция ниже поможет вам с генерацией подвыборок (батчей).

In [ ]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    batch = []
    neg_samples = []

    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [ ]:
batch_size = 4
num_negatives = 15
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

Наконец, время реализовать модель. Обращаем ваше внимание, использование линейных слоев (`nn.Linear`) далеко не всегда оправданно!

Напомним, что в случае negative sampling решается задача максимизации следующего функционала:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

где:
- $\mathbf{v}_{w_I}$ – вектор центрального слова $w_I$,
- $\mathbf{v}'_{w_O}$ – вектор слова из контекста $w_O$,
- $k$ – число negative samplesЮ,
- $P_n(w)$ – распределение negative samples, заданное выше,
- $\sigma$ – сигмоида.

In [ ]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)# YOUR CODE HERE
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)  # YOUR CODE HERE

        nn.init.normal_(self.center_embeddings.weight, mean = 0, std = 0.01)
        nn.init.normal_(self.context_embeddings.weight, mean = 0, std = 0.01)


    def forward(self, center_words, pos_context_words, neg_context_words):
        # YOUR CODE HERE
        center_emb = self.center_embeddings(center_words)
        pos_cont_emb = self.context_embeddings(pos_context_words)
        neg_cont_emb = self.context_embeddings(neg_context_words)

            # Позитивные оценки
        pos_scores = torch.mul(center_emb, pos_cont_emb).sum(dim=1)  # (batch_size,)

        # Негативные оценки, без суммирования по негативным примерам
        neg_scores = torch.bmm(center_emb.unsqueeze(1), neg_cont_emb.transpose(1,2)).squeeze()  # (batch_size, num_negatives)


        return pos_scores, neg_scores

In [ ]:
device = torch.device("cpu")

In [ ]:
vocab_size = len(word_to_index)
embedding_dim = 32
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [ ]:
for step in tqdma(range(10)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
neg_scores.shape

torch.Size([512, 15])

In [ ]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()

        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [ ]:
steps = 2500
batch_size = 512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

  0%|          | 0/2500 [00:00<?, ?it/s]

Step 0, Loss: 1.3863070011138916, learning rate: [0.05]
Step 100, Loss: 1.4351658117771149, learning rate: [0.05]
Step 200, Loss: 1.6491793060302735, learning rate: [0.025]
Step 300, Loss: 1.6167073011398316, learning rate: [0.025]
Step 400, Loss: 1.5552930796146394, learning rate: [0.0125]
Step 500, Loss: 1.5192208039760589, learning rate: [0.00625]
Step 600, Loss: 1.495102708339691, learning rate: [0.00625]
Step 700, Loss: 1.465018345117569, learning rate: [0.003125]
Step 800, Loss: 1.4479576253890991, learning rate: [0.003125]
Step 900, Loss: 1.4374318850040435, learning rate: [0.003125]
Step 1000, Loss: 1.4356036055088044, learning rate: [0.003125]
Step 1100, Loss: 1.4146727323532104, learning rate: [0.003125]
Step 1200, Loss: 1.4099740397930145, learning rate: [0.003125]
Step 1300, Loss: 1.3991815161705017, learning rate: [0.0015625]
Step 1400, Loss: 1.3958247339725494, learning rate: [0.0015625]
Step 1500, Loss: 1.3858061170578002, learning rate: [0.0015625]
Step 1600, Loss: 1.38

Наконец, используйте полученную матрицу весов в качестве матрицы в векторными представлениями слов. Рекомендуем использовать для сдачи матрицу, которая отвечала за слова из контекста (т.е. декодера).

In [ ]:
_model_parameters = model.parameters()
embedding_matrix_center = next(
    _model_parameters
).detach()
embedding_matrix_context = next(
    _model_parameters
).detach()


In [ ]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

In [ ]:
embedding_matrix_context

tensor([[ 0.3845, -1.1000, -0.0958,  ..., -0.1899,  0.9514, -0.4601],
        [-0.8066, -0.4683, -0.1674,  ..., -0.2918, -0.4602, -0.6457],
        [-0.1493, -0.3525, -0.6998,  ..., -0.6207, -0.3680,  0.4679],
        ...,
        [ 0.7728,  1.0210, -0.0805,  ...,  0.1606,  0.5417,  0.1140],
        [-1.2277, -0.9466, -0.5551,  ..., -0.1342,  1.3691,  0.0055],
        [-0.8161, -1.2861, -0.8678,  ..., -1.7712, -0.5274, -0.4010]])

In [ ]:
len(word_to_index)

28546

Простые проверки:

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("dell", embedding_matrix_context)[None, :],
)
similarity_1.item() > similarity_2.item()


True

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)
similarity_1.item()

0.09397971630096436

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

Наконец, взглянем на ближайшие по косинусной мере слова. Функция реализована ниже.

In [ ]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [ ]:
find_nearest("python", embedding_matrix_context, k=10)

[('define', 0.635047972202301),
 ('helical', 0.6361266374588013),
 ('bbmp', 0.6406078338623047),
 ('spells', 0.642306387424469),
 ('5star', 0.6522794961929321),
 ('shoud', 0.6554765105247498),
 ('revamp', 0.661026656627655),
 ('constable', 0.664386510848999),
 ('lightsabers', 0.6685476899147034),
 ('python', 1.0000001192092896)]

Также вы можете визуально проверить, как представлены в латентном пространстве часто встречающиеся слова.

In [ ]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

tensor([[-3.2366e-02, -6.0896e-02,  1.1420e+00, -7.6141e-01, -5.8354e-01,
          5.0486e-01,  1.2845e-01, -1.7405e+00,  7.2240e-01,  3.0446e-01,
          3.4879e-02, -3.9526e-01,  4.4869e-02, -1.2255e-01,  3.2545e-02,
         -1.3061e-01, -4.5475e-01,  2.0235e-01,  1.1598e+00,  5.8524e-01,
          5.1494e-01,  3.0741e-01,  2.2906e-01,  5.7710e-01,  4.0859e-01,
          1.3529e+00,  3.2671e-02,  3.5430e-01,  3.5991e-01,  1.0619e+00,
          3.9490e-01,  1.9319e-01],
        [ 1.6110e+00, -1.4538e+00,  1.5691e+00,  2.7544e-01, -3.0814e-01,
          1.5793e+00,  1.2696e-01,  1.9717e-01,  1.4486e+00,  6.4536e-01,
          8.7415e-01,  1.1334e+00,  1.5370e+00, -1.5074e+00, -1.4271e+00,
          1.8802e-01, -5.6599e-01,  5.2021e-01,  1.3025e+00,  1.7595e-01,
         -1.2529e-02, -7.6303e-01, -2.4659e-01, -6.8123e-01, -4.3090e-01,
          1.5446e+00, -1.4209e+00,  1.2396e+00, -1.4225e+00, -3.0913e-01,
         -6.4507e-01, -1.5888e+00],
        [ 1.4000e-01,  3.6865e-01,  1.49

In [ ]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [ ]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

In [ ]:
import umap

In [ ]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

AttributeError: module 'umap' has no attribute 'UMAP'

In [ ]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

NameError: name 'embedding' is not defined

Для сдачи задания необходимо загрузить функции `subsample_frequent_words` и `get_negative_sampling_prob`, а также сгенерировать файл для посылки ниже и приложить в соответствующую задачу. Успехов!

In [ ]:
# do not change the code in the block below
# __________start of block__________
import os
import json

assert os.path.exists(
    "/content/words_subset (1).txt"
), "Please, download `words_subset.txt` and place it in the working directory"

with open("/content/words_subset (1).txt") as iofile:
    selected_words = iofile.read().split("\n")


def get_matrix_for_selected_words(selected_words, embedding_matrix, word_to_index):
    word_vectors = []
    for word in selected_words:
        index = word_to_index.get(word, None)
        vector = [0.0] * embedding_matrix.shape[1]
        if index is not None:
            vector = embedding_matrix[index].numpy().tolist()
        word_vectors.append(vector)
    return word_vectors


word_vectors = get_matrix_for_selected_words(
    selected_words, embedding_matrix_context, word_to_index
)

with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
    iofile.write(" ")
print("File saved to `submission_dict.json`")
# __________end of block__________

File saved to `submission_dict.json`


[[-0.5000515580177307,
  0.015528488904237747,
  -0.3867218792438507,
  -0.193895161151886,
  -0.2508697509765625,
  0.05095751956105232,
  -0.048809949308633804,
  -0.2004501223564148,
  -0.8284247517585754,
  -0.33480915427207947,
  0.265392005443573,
  0.17404140532016754,
  0.052019719034433365,
  0.6502013206481934,
  0.06176205351948738,
  0.3185977637767792,
  -0.30176353454589844,
  -0.3264741599559784,
  -0.412211149930954,
  -0.01722096838057041,
  0.37156766653060913,
  -0.12589840590953827,
  -0.0639808177947998,
  -0.1219809427857399,
  0.1383918821811676,
  0.773907482624054,
  -0.13088330626487732,
  -0.12434958666563034,
  0.37086549401283264,
  0.04848053306341171,
  -0.05162927508354187,
  -0.4671548902988434],
 [-0.023456647992134094,
  -0.04679768159985542,
  0.13420802354812622,
  -0.016456864774227142,
  0.05550625920295715,
  -0.27456986904144287,
  -0.23206880688667297,
  -0.16962967813014984,
  0.24033670127391815,
  -0.1793423891067505,
  -0.11310955882072449,